# Импорт библиотек

In [1]:
from io import BytesIO
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image, ImageStat
import io
import torch
from torchvision import transforms
import os
import random
from sklearn.model_selection import train_test_split

# Подготовка данных trashnet_enhanced к обучению



## Загрузка данных

In [2]:
# Функция для загрузки датасета в формате parquet
def get_df_from_hf_parquet(base_url: str, parquet_count: int) -> pd.DataFrame:
  urls = [base_url + f"{i:04}.parquet" for i in range(0, parquet_count)]
  return pd.concat([pd.read_parquet(url) for url in urls])

In [4]:
enhanced_url = "https://huggingface.co/datasets/edwinpalegre/trashnet_enhanced/resolve/refs%2Fconvert%2Fparquet/default/train/"

df_enhanced = get_df_from_hf_parquet(enhanced_url, 100)

удаление прошлых данных

In [5]:
#!rm -rf /content/resized_images
#!rm -rf /content/augmented_images

## Названия лейблов
0. biodegradable
1. cardboard
2. glass
3. metal
4. paper
5. plastic
6. trash (мусор без категории)

## Информация о dataset

In [7]:
df_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19892 entries, 0 to 197
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   19892 non-null  object
 1   label   19892 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 466.2+ KB


In [8]:
print(df_enhanced.head())

                                               image  label
0  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      0
1  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      0
2  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      0
3  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      0
4  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      0


# Балансировка данных

## Масштабирование размера изображений и сохранение в папку "resized_images"

Исходя из EDA самым популярным размером стал 416 на 416. Приводим все изображения к кваратному виду 416 на 416, путем их растягивания.


In [9]:

# Создаём папку для сохранения
SAVE_DIR = "resized_images"
os.makedirs(SAVE_DIR, exist_ok=True)

# Функция для изменения размера и сохранения
def resize_and_save(row, size=(416, 416)):
    img_bytes = row["image"]["bytes"]
    img = Image.open(io.BytesIO(img_bytes))
    img_resized = img.resize(size)  # Меняем размер
    img_path = os.path.join(SAVE_DIR, f"{row.name}.jpg")  # Генерируем путь
    img_resized.save(img_path, "JPEG")
    return img_path


df_enhanced["image_path"] = df_enhanced.apply(resize_and_save, axis=1)
#содержит пути к уменьшенным изображениям


## Балансировка количества объектов в классах

### Агументация класса trash

Количество изображений в классах

In [10]:
print(df_enhanced["label"].value_counts())

label
2    4902
0    3692
4    3485
1    2782
3    2484
5    2410
6     137
Name: count, dtype: int64


Папка для агументированных изображений

In [11]:
SAVE_DIR = "augmented_images"
os.makedirs(SAVE_DIR, exist_ok=True)

Агументация будет производится с изменением яркости, констрастности, поворотов и отражений

In [12]:
target_size = df_enhanced["label"].value_counts().max()

# функция агументации
def augment_image(img):
    angle = random.choice([90, 180, 270])
    transform = transforms.Compose([
        transforms.ColorJitter(brightness=0.3, contrast=0.3), # регулировка яркости и контрастности
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Lambda(lambda x: x.rotate(angle))
    ])
    return transform(img)


augmented_data = []


for class_label, count in df_enhanced["label"].value_counts().items():
    df_class = df_enhanced[df_enhanced["label"] == class_label]
    n_samples_needed = target_size - count  # сколько нужно добавить

    if n_samples_needed > 0:
        for i in range(n_samples_needed):
            row = df_class.sample(n=1, random_state=i).iloc[0]
            img = Image.open(row["image_path"])
            img_aug = augment_image(img)

            # Сохраняем аугментированное изображение на диск
            aug_path = os.path.join(SAVE_DIR, f"aug_{class_label}_{i}.jpg")
            img_aug.save(aug_path, "JPEG")

            augmented_data.append({"image_path": aug_path, "label": class_label})

Объединяем с исходным dataset

In [13]:
df_augmented = pd.DataFrame(augmented_data)
df_balanced = pd.concat([df_enhanced, df_augmented], ignore_index=True)

Количество изображений в классах после агументации

In [14]:
print(df_balanced["label"].value_counts())

label
0    4902
1    4902
2    4902
3    4902
4    4902
5    4902
6    4902
Name: count, dtype: int64


# Разделение данных на обучающие и тестовую выборки

In [15]:
train_df, test_df = train_test_split(df_balanced, test_size=0.2, random_state=42, stratify=df_balanced["label"])

In [16]:
print("Размер обучающей выборки:", train_df.shape)
print("Размер тестовой выборки:", test_df.shape)

Размер обучающей выборки: (27451, 3)
Размер тестовой выборки: (6863, 3)


In [17]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 27451 entries, 9546 to 32636
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       15920 non-null  object
 1   label       27451 non-null  int64 
 2   image_path  27451 non-null  object
dtypes: int64(1), object(2)
memory usage: 857.8+ KB
None


- 80% обучающая выборка
- 20% тестовая выборка
- с равномерным распределением классов в выборках